In [1]:

from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

from sklearn.model_selection import train_test_split

2022-07-08 18:01:23.722421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-08 18:01:23.722441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
IMG_SIZE = 256
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048


In [3]:
pathtoData="../results/segmented_videos/"

df=pd.read_csv(pathtoData+"metadata.csv",index_col=False)
print(df)
df=df[{"name","class"}]
print(df)
X_train, X_test, y_train, y_test = train_test_split(df["name"],df["class"], test_size=0.15,stratify=df["class"])

     Unnamed: 0.1  Unnamed: 0  name class     t0
0               0           0    V1     H  23.75
1               1           1    V2     H  23.75
2               2           2    V3     A  23.75
3               3           3    V4     F  23.75
4               4           4    V5     H  23.75
..            ...         ...   ...   ...    ...
272           272         272  V273     G  21.25
273           273         273  V274     H  21.50
274           274         274  V275     E  21.50
275           275         275  V276     D  19.50
276           276         276  V277     D  23.25

[277 rows x 5 columns]
     name class
0      V1     H
1      V2     H
2      V3     A
3      V4     F
4      V5     H
..    ...   ...
272  V273     G
273  V274     H
274  V275     E
275  V276     D
276  V277     D

[277 rows x 2 columns]


/tmp/ipykernel_724671/1989333073.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df=df[{"name","class"}]


In [4]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

print(f"Total videos for training: {len(X_train)}")
print(f"Total videos for testing: {len(X_test)}")

(235,) (235,)
(42,) (42,)
Total videos for training: 235
Total videos for testing: 42


In [5]:
def load_video(path, max_frames=300, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [6]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    
    return keras.Model(inputs, outputs, name="feature_extractor")
feature_extractor = build_feature_extractor()

2022-07-08 18:01:26.505664: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 18:01:26.505945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mmeignin/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-07-08 18:01:26.506037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mmeignin/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-07-08 18:01:26.506115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dler

In [7]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(y_train)
)
print(label_processor.get_vocabulary())


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']


In [8]:
train_df= pd.concat([X_train,y_train],axis=1)
test_df= pd.concat([X_test,y_test],axis=1)
print(train_df.shape,test_df.shape)



(235, 2) (42, 2)


In [9]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = (pathtoData+df["name"].values+".mp4").tolist()
    labels = df["class"].values
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(path)
        # Gather all its frames and add a batch dimension.
        frames = load_video( path)
        frames = frames[None, ...]


        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )

            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels




In [10]:
train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"Frame features in train set: {test_data[0].shape}")
print(f"Frame masks in train set: {test_data[1].shape}")

../results/segmented_videos/V60.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V96.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V11.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V1.mp4
1/1 [==============================] - 0s 72ms/step
../results/segmented_videos/V240.mp4
1/1 [==============================] - 0s 56ms/step
../results/segmented_videos/V31.mp4
1/1 [==============================] - 0s 54ms/step
../results/segmented_videos/V106.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V99.mp4
1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V180.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V59.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V139.mp4
1/1 [==============================] - 0s 48ms/step
../results/segmented_videos/V261.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V170.mp4
1/1 [==============================] - 0s 49ms/step
../results/segmented_videos/V50.mp4
1/1 [==============================] - 0s 57ms/step
../results/segmented_videos/V135.mp4
1/1 [==============================] - 0s 49ms/step
../results/segmented_videos/V248.mp4
1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V43.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V123.mp4
1/1 [==============================] - 0s 54ms/step
../results/segmented_videos/V160.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V136.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V52.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V6.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V211.mp4
1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V168.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V171.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V152.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V145.mp4
1/1 [==============================] - 0s 49ms/step
../results/segmented_videos/V243.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V198.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V19.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V51.mp4
1/1 [==============================] - 0s 56ms/step


1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V191.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V132.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V105.mp4
1/1 [==============================] - 0s 57ms/step
../results/segmented_videos/V41.mp4
1/1 [==============================] - 0s 62ms/step
../results/segmented_videos/V30.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V93.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V7.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V102.mp4


...

1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V86.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V122.mp4
1/1 [==============================] - 0s 56ms/step
../results/segmented_videos/V125.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V84.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V227.mp4
1/1 [==============================] - 0s 48ms/step
../results/segmented_videos/V208.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V178.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V61.mp4
1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 73ms/step
../results/segmented_videos/V94.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V29.mp4
../results/segmented_videos/V232.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V85.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V172.mp4
1/1 [==============================] - 0s 62ms/step
../results/segmented_videos/V167.mp4
1/1 [==============================] - 0s 55ms/step
../results/segmented_videos/V21.mp4
../results/segmented_videos/V202.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V78.mp4
1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V194.mp4
1/1 [==============================] - 0s 55ms/step
../results/segmented_videos/V183.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V118.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V40.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V127.mp4
1/1 [==============================] - 0s 51ms/step
../results/segmented_videos/V150.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V258.mp4
1/1 [==============================] - 0s 60ms/step
../results/segmented_videos/V242.mp4
1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 49ms/step
../results/segmented_videos/V87.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V223.mp4
1/1 [==============================] - 0s 56ms/step
../results/segmented_videos/V203.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V206.mp4
1/1 [==============================] - 0s 48ms/step
../results/segmented_videos/V234.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V251.mp4
1/1 [==============================] - 0s 55ms/step
../results/segmented_videos/V124.mp4
1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 54ms/step
../results/segmented_videos/V275.mp4
1/1 [==============================] - 0s 59ms/step
../results/segmented_videos/V184.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V155.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V14.mp4
1/1 [==============================] - 0s 50ms/step
../results/segmented_videos/V88.mp4
1/1 [==============================] - 0s 52ms/step
../results/segmented_videos/V42.mp4
1/1 [==============================] - 0s 53ms/step
../results/segmented_videos/V257.mp4
1/1 [==============================] - 0s 50ms/step
Frame features in train set: (235, 20, 2048)
Frame masks in train set: (235, 20)
Frame features in train set: (42, 20, 2048)
Frame masks in train set: (42, 20)


In [136]:
from sklearn.metrics import confusion_matrix
def wce( y_true, y_pred):
        print(y_true)
        n_classes = 8.
        W = tf.constant(
            [
                [0, 1, 6, 10, 10, 10, 10, 10],
                [1, 0, 3, 10, 10, 10, 10, 10],
                [6, 3, 0, 2, 9, 10, 10, 10],
                [10, 10, 2, 0, 9, 9, 10, 10],
                [10, 10, 9, 9, 0, 8, 8, 8],
                [10, 10, 10, 9, 8, 0, 9, 8],
                [10, 10, 10, 10, 8, 9, 0, 9],
                [10, 10, 10, 10, 8, 8, 9, 0],
                
            ],shape=(8,8),dtype=tf.float32
        )
        W = W / tf.math.reduce_max(W)
        # Convert proba to hard-labels
        y_pred = tf.math.argmax(y_pred)
        y_T=tf.Variable(tf.zeros([8],tf.int64))
        y_T=y_T[tf.math.reduce_max(y_true)].assign(1)

        print(y_T,y_pred)
        n = len(y_true)
        conf_mat=tf.math.confusion_matrix(y_true, y_pred,dtype=tf.float32)
        loss = tf.math.reduce_sum(tf.linalg.matmul(conf_mat, W)) / 8
        print(loss)

        return loss



In [137]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)


    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss=wce, optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()


Epoch 1/100
Tensor("IteratorGetNext:2", shape=(None, 1), dtype=int64)
<tf.Variable 'UnreadVariable' shape=(8,) dtype=int64> Tensor("wce/ArgMax:0", shape=(8,), dtype=int64)
Tensor("wce/truediv_1:0", shape=(), dtype=float32)


ValueError: in user code:

    File "/home/mmeignin/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/mmeignin/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/mmeignin/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/mmeignin/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 893, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/mmeignin/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/optimizer_v2.py", line 539, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "/home/mmeignin/anaconda3/lib/python3.9/site-packages/keras/

In [ ]:
sequence_model.summary()

In [38]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames





test_video = np.random.choice(test_df["name"].values)
print(test_df.where(test_df["name"]==test_video).dropna())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)



     name class
205  V206     G
Test video path: V206
1/1 [==============================] - 0s 14ms/step
  H: 15.73%
  F: 13.12%
  C: 12.79%
  B: 12.33%
  A: 11.97%
  G: 11.92%
  E: 11.68%
  D: 10.46%
